In [1]:
import os
import google.oauth2.credentials
import pickle
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [2]:
credentials = None

In [3]:
if os.path.exists('yt_token.pickle'):
    with open('yt_token.pickle', 'rb') as t:
        credentials = pickle.load(t)

In [ ]:
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        credentials.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secret.json',
            scopes = [
                'https://www.googleapis.com/auth/youtube.readonly'
            ]
        )

        flow.run_local_server(port=8080, prompt='consent')
        credentials = flow.credentials

        with open('yt_token.pickel', 'wb') as t:
            pickle.dump(credentials, t)

In [ ]:
yt_service = build('youtube', 'v3', credentials=credentials)


In [ ]:
#yt_service = build('youtube', 'v3', credentials=credentials)

#request = yt_service.playlistItems().list()

In [ ]:
#yt_service.close()